In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
# Read into cities csv created in WeatherPy
file_to_load= ('../output_data/cities.csv')
cities_df= pd.read_csv(file_to_load)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ponta do sol,32.67,-17.10,73.40,78,40,2.24,PT,1597870747
1,eyl,7.98,49.82,78.71,75,0,20.62,SO,1597870747
2,busselton,-33.65,115.33,41.00,79,81,1.05,AU,1597870637
3,northfield,44.46,-93.16,82.40,54,1,11.41,US,1597870749
4,albany,42.60,-73.97,73.99,42,17,7.31,US,1597870730


In [4]:
# Humidity Heatmap
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]].astype(float)
humidity= cities_df['Humidity'].astype(float)

fig= gmaps.figure()
heat_layer= gmaps.heatmap_layer(locations, weights=humidity, 
                             dissipating=False, max_intensity=100, 
                             point_radius= 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Narrow Dataframe to ideal weather conditions
ideal_weather_df= cities_df.loc[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80) 
                                & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]
ideal_weather_df= ideal_weather_df.dropna(how='any')
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
101,la rioja,42.25,-2.50,71.60,56,0,1.12,ES,1597870814
121,durusu,41.31,28.68,75.99,73,0,5.82,TR,1597870827
147,shahrud,36.42,54.98,71.17,46,0,4.59,IR,1597870847
151,virden,39.50,-89.77,79.00,45,0,4.00,US,1597870690
162,itaocara,-21.67,-42.08,79.88,46,0,2.59,BR,1597870858


In [6]:
hotel_name= []

for index, row in ideal_weather_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results= response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_name.append(response['results'][0]['name'])
    except:
        print("Missing field/result... skipping.")
        
    
hotel_df= ideal_weather_df        
hotel_df["Hotel Name"] = hotel_name
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

Closest hotel is San Román de Cameros.
Closest hotel is Karaburun.
Closest hotel is Shahroud.
Closest hotel is Virden.
Closest hotel is Itaocara.
Closest hotel is Marbella.
Closest hotel is Hà Giang.
Closest hotel is Panguna.
Closest hotel is Kamaishi.
Closest hotel is Alexandria.
Closest hotel is Campanha.
Closest hotel is Kilis.
Closest hotel is Suez.
Closest hotel is Donskoye.
Closest hotel is Zhanaozen.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
101,la rioja,42.25,-2.50,71.60,56,0,1.12,ES,1597870814,San Román de Cameros
121,durusu,41.31,28.68,75.99,73,0,5.82,TR,1597870827,Karaburun
147,shahrud,36.42,54.98,71.17,46,0,4.59,IR,1597870847,Shahroud
151,virden,39.50,-89.77,79.00,45,0,4.00,US,1597870690,Virden
162,itaocara,-21.67,-42.08,79.88,46,0,2.59,BR,1597870858,Itaocara


In [8]:
info_box= """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info= [info_box.format(**row) for index,row in hotel_df.iterrows()]
locations= hotel_df[['Lat', 'Lng']]
                     
markers= gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))